In [24]:
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras_vggface.vggface import VGGFace
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from seaborn import heatmap
###            ###
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import os
import cv2

## PARAMETERS ##
print_tensorflow_GPU_info = False
tensorflow_verbosity = "INFO"  # DEBUG(10): All | INFO(20): Info&Warning | WARN(30)[Default]: Warning | ERROR(40): Error | FATAL(50): None
random_state = 22
    # General
dataset_name = "JAFFE"
multiclass = True
num_features = 64
num_labels = 7
width, height = 197, 197
channels = 3
load_npy = True
data_generator = False
    # Convolution
kernel_size = (3, 3)
pool_size = (2, 2)
strides = (2, 2)
dropout = 0.3
    # Training
batch_size = 64
epochs = 10
loss = "categorical_crossentropy"
optimizer = "adam"
eval_metrics = "accuracy"
##             ##

## Reproducibility ## 
random.seed(random_state)  # Python's seed
np.random.seed(random_state)  # Numpy's seed
tf.set_random_seed(random_state)  # Tensorflow's seed
##                 ##

In [25]:
## GPU usage settings for Tensorflow backend ##
# RTX GPU Memory BUG Fix & Must also be placed at the top of the code else it doesn't work
from keras.backend import tensorflow_backend as K
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True                     # dynamically grow the memory used on the GPU
#tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9  # fraction of the GPU to be used
#tf_config.log_device_placement = True                        # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=tf_config)
K.set_session(sess)                     # set this TensorFlow session as the default session for Keras
##                                                                                          ##     

## Tensorflow Verbosity Module ##
default_verbosity = tf.compat.v1.logging.get_verbosity()
tf.compat.v1.logging.set_verbosity(tensorflow_verbosity)
print(f"\n--CHANGED TENSORFLOW VERBOSITY FROM {default_verbosity/10:.0f} (default) TO {tf.compat.v1.logging.get_verbosity()/10:.0f}")
##                             ##

## Tensorflow GPU Information Module ##
if print_tensorflow_GPU_info == True:
    print(f"\n--AVAILABLE GPUS:")
    K._get_available_gpus()
    print(f"\n--NUM OF GPUs AVAILABLE: {len(tf.config.experimental.list_physical_devices('GPU'))}")
    print(f"\n--IS TF BUILT WITH CUDA: {tf.test.is_built_with_cuda()}")
    print(f"\n--IS GPU AVAILABLE: {tf.test.is_gpu_available()}")
##                                   ##  


--CHANGED TENSORFLOW VERBOSITY FROM 2 (default) TO 2


# Load JAFFE

In [32]:
if dataset_name == "JAFFE":
    # Load images
    data_path = "./Datasets/Japanese Female Facial Expression (JAFFE) Dataset/PNG Format"    
    data_dir_list = os.listdir(data_path)

    img_data_list=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{} with length {}'.format(dataset, len(img_list)))
        for img in img_list:
            input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img, cv2.IMREAD_COLOR)  # Dataset is actually Grayscale
            input_img=cv2.resize(input_img, (197,197))  # RESIZE Required
            img_data_list.append(input_img)
            
    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    #X = img_data/255

    # Make image VGG compatible (3 channels instead of 1 channel gray scale, BGR, etc.)
    X_new = np.empty((len(X), width, height, channels))
    print(X[1])
    for i in range(len(X)):
        X_new[i] = preprocess_input(X[i], mode='caffe')
    X = X_new
    print(X_new[1])    
    del X_new
    del img_data

    print(X.shape, "\n")

    # Define the number of classes
    num_of_samples = X.shape[0]
    labels = np.ones((num_of_samples,), dtype='int64')
    labels[0:30]=0 #30
    labels[30:59]=1 #29
    labels[59:91]=2 #32
    labels[91:122]=3 #31
    labels[122:152]=4 #30
    labels[152:183]=5 #31
    labels[183:]=6 #30  

    # Class Label Names    
    label_names = ['ANGRY', 'DISGUST', 'FEAR', 'HAPPY', 'NEUTRAL', 'SAD', 'SURPRISE']

    # Convert class labels to on-hot encoding 
    y = to_categorical(labels, num_labels)

    # Shuffle the dataset
    X, y = shuffle(X, y, random_state=random_state)
    print("Loading Done")

elif dataset_name == "SFEW":
    # Load images from Train Folder
    data_path = "./Datasets/Static Facial Expression In The Wild (SFEW) Dataset/SFEW_2/Train"    
    data_dir_list = os.listdir(data_path)

    img_data_list=[]
    labels=[]
    count = 0

    for dataset in data_dir_list:
        if (dataset.endswith(".zip") != True):
            img_list=os.listdir(data_path+'/'+ dataset)
            print ('Loaded the images of dataset-'+'{} with length {}'.format(dataset, len(img_list)))
            for img in img_list:
                input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img, cv2.IMREAD_COLOR)  # Dataset is RGB
                input_img = input_img[:, 72:648]  # CROP Required
                input_img=cv2.resize(input_img, (48,48))  # RESIZE Required
                img_data_list.append(input_img)
                labels.append(count)
            count += 1

    # Also images from Val Folder
    data_path = "./Datasets/Static Facial Expression In The Wild (SFEW) Dataset/SFEW_2/Val"    
    data_dir_list = os.listdir(data_path)

    count = 0

    for dataset in data_dir_list:
        if (dataset.endswith(".zip") != True):
            img_list=os.listdir(data_path+'/'+ dataset)
            print ('Loaded the images of dataset-'+'{} with length {}'.format(dataset, len(img_list)))
            for img in img_list:
                input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img, cv2.IMREAD_COLOR)  # Dataset is RGB
                input_img = input_img[:, 72:648]  # CROP Required
                input_img=cv2.resize(input_img, (48,48))  # RESIZE Required
                img_data_list.append(input_img)
                labels.append(count)
            count += 1

    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    X = img_data/255

    # Make image VGG compatible (3 channels instead of 1 channel gray scale, BGR, etc.)
    X_new = np.empty((len(X), width, height, channels))
    for i in range(len(X)):
        X_new[i] = preprocess_input(X[i], mode='caffe')
    X = X_new
    del X_new
    del img_data

    print(X.shape, "\n")
    labels = np.asarray(labels, dtype='int64')

    # Class Label Names    
    label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

    # Convert class labels to on-hot encoding 
    y = to_categorical(labels, num_labels)

    # Shuffle the dataset
    X, y = shuffle(X, y, random_state=random_state)
    print("Loading Done")
    print(len(X))    

Loaded the images of dataset-ANGRY with length 30
Loaded the images of dataset-DISGUST with length 29
Loaded the images of dataset-FEAR with length 32
Loaded the images of dataset-HAPPY with length 31
Loaded the images of dataset-NEUTRAL with length 30
Loaded the images of dataset-SAD with length 31
Loaded the images of dataset-SURPRISE with length 30
[[[-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  ...
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]]

 [[-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  ...
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]]

 [[-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  ...
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.68 ]]

 ...

 [[-103.939 -116.779 -123.68 ]
  [-103.939 -116.779 -123.6

# Load Model

In [27]:
''' Load Model '''
model = load_model('Saved Models/GitHub-FER_ResNet50_VGGFace.h5')

# Evaluate on JAFFE (need to also make sure that labels match the FER dataset)

In [28]:
X_test = X
y_test = y

In [29]:
''' Evaluate Model '''
results = model.evaluate(X_test, y_test)
print()

for i in range(len(model.metrics_names)):
    print("Metric - {}: {:.5f}".format(model.metrics_names[i], results[i]*100))

if multiclass == False: 
    predictions = [1 * (x[0]>=0.5) for x in model.predict(X_test)]
    cm = confusion_matrix(y_test, predictions)
else:    
    predictions = np.argmax(model.predict(X_test), axis=1)
    cm = confusion_matrix(y_test.argmax(axis=1), predictions)
print()
print(cm)

213/213 [==============================] - 2s 8ms/step

Metric - loss: 28916.83825
Metric - accuracy: 14.08451

[[ 0  0  0  0 30  0  0]
 [ 0  0  0  0 29  0  0]
 [ 0  0  0  0 32  0  0]
 [ 0  0  0  0 31  0  0]
 [ 0  0  0  0 30  0  0]
 [ 0  0  0  0 31  0  0]
 [ 0  0  0  0 30  0  0]]
